In [193]:
# imports
import torch
from torch.utils.data import Dataset, DataLoader, random_split, SequentialSampler, RandomSampler, BatchSampler
from torchvision import models, transforms
from torch import nn, optim

from PIL import Image
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from utils.clean_labels import clean_labels
from utils.prepare_images import prepare_images
from utils.mydataset import SVRCDataset

import os
from datetime import datetime
import random
import time

%load_ext autoreload
%autoreload 2
print(torch.version.cuda)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
None


In [197]:
# put videos here!
video_base = 'data/videos'
#video_base = 'D:/e6691/6691_assignment2/videos'
videos = [vid for vid in os.listdir(video_base) if vid.endswith('mp4')]
# images will be output to here
image_base = 'data/images'
#image_base = 'D:/e6691/6691_assignment2/images'
if not os.path.exists(image_base):
    os.mkdir(image_base)

In [ ]:
#!rm -r {image_base + '/*'}

In [199]:
videos

['RALIHR_surgeon01_fps01_0002.mp4', 'RALIHR_surgeon01_fps01_0001.mp4']

In [201]:
# command line: 
# ffmpeg -i {input_video} -r {frame_rate} [-f {force_format} (not needed)] {output_images}
# doc: https://ffmpeg.org/ffmpeg.html
for video in videos:
    input_path = os.path.join(video_base, video)
    # make dirs
    output_base = image_base + '/{}'.format(video.split('.')[0])
    if not os.path.exists(output_base):
        os.mkdir(output_base)
    output_path = os.path.join(output_base, '%d.png')
    # # command
    print('Frames extracted from {} to {}'.format(input_path, output_path))
    !ffmpeg -i {input_path} -r 1 {output_path}

Frames extracted from data/videos/RALIHR_surgeon01_fps01_0002.mp4 to data/images/RALIHR_surgeon01_fps01_0002/%d.png
ffmpeg version 4.3 Copyright (c) 2000-2020 the FFmpeg developers
  built with clang version 10.0.0
  configuration: --prefix=/Users/harrypotter/opt/anaconda3/envs/dl --cc=x86_64-apple-darwin13.4.0-clang --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'data/videos/RALIHR_surgeon01_fps01_0002.mp4':
  Metadata:
  

In [202]:
labels_path = 'data/labels/video.phase.trainingData.clean.StudentVersion.csv'
names_path = 'data/labels/all_labels_hernia.csv'
labels_df = pd.read_csv(labels_path)
names_df = pd.read_csv(names_path, index_col='id')

In [203]:
names_df

,labels
id,
0,adhesiolysis
1,blurry
2,catheter insertion
3,mesh placement
4,mesh positioning
5,out of body
6,peritoneal closure
7,peritoneal scoring
8,positioning suture


In [204]:
prepare_images(video_base, image_base, labels_df, names_df, 'png')

/Users/harrypotter/Documents/courses/dl/assignment2/e6691-2022spring-assign2-vcsz/utils/prepare_images.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  video_df[['StartSec', 'EndSec']] = video_df[['Start', 'End']].applymap(time2int)
/Users/harrypotter/Documents/courses/dl/assignment2/e6691-2022spring-assign2-vcsz/utils/prepare_images.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  video_df[['StartSec', 'EndSec']] = video_df[['Start', 'End']].applymap(time2int)
/Users/harrypotter/Documents/courses/d